# Author:

- Huu Khang Nguyen - 7402909
- hkn878@uowmail.edu.au


# Environment:

- Python 3.10.8
- Ubuntu 22.04.2 LTS x86_64


# Task:

- Create 3 Decision Tree (DT) models to predict the application ranking with the following split criterias respectively:
  - Information Gain
  - Gain Ratio
  - Gini Index (Gini impurity)
- Afterward, build a random forest classifier from created 3 DT models


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/nursery.data', names=[
    'parents',
    'has_nurs',
    'form',
    'children',
    'housing',
    'finance',
    'social',
    'health',
    'application ranking'
])

In [3]:
df.head()

,parents,has_nurs,form,children,housing,finance,social,health,application ranking
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority


In [4]:
df.shape

(12960, 9)

# Preprocessing the data


## Droping rows with NA values (if any)


In [5]:
df.dropna()

,parents,has_nurs,form,children,housing,finance,social,health,application ranking
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
...,...,...,...,...,...,...,...,...,...
12955,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,priority,spec_prior
12956,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,not_recom,not_recom
12957,great_pret,very_crit,foster,more,critical,inconv,problematic,recommended,spec_prior
12958,great_pret,very_crit,foster,more,critical,inconv,problematic,priority,spec_prior


## Splitting the dataset


In [6]:
TRAINING_SPLIT = 0.6 # 60%
shuffle_df = df.sample(frac=1)

train_size = int(TRAINING_SPLIT * len(df))

y = shuffle_df['application ranking']
X = shuffle_df.drop(columns=['application ranking'])

X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

In [7]:
len(X_test)

5184

# Implementing the Decision Tree model


I will implement the decision tree model with the following properties:
- **Binary split** decision tree
- Early stopping critrias to improve training speed & prevent overfitting:
    - When all records in the sample share the same class label (or number of unique class label equals 1)
    - Minimum sample split: stop the splitting if the node contains fewer samples than this minimum
    - Max depth of the tree: restrict the height of the tree
    
    *The node with the most common class label will be return when early stopping occured (base case)*

Overall, when fitting the dataset into the decision tree, we will:
- Recursively generating the decision tree by getting the most important feature to split the data. 
    - This is done by calculating the best splitting criterion (which can be information gain, gini index, or gain ratio, depending on what user chose as criterion for the classifier) for each feature.
- Divide the data based on the selected splitting feature into two subsets.
- The process will then be repeated for each subset until stopping criteria (as defined above) is reached. A leaf node will be added into the tree, with its `value` as the most common label across all the samples in that node

When predict a dataset with decision tree, the tree is traversed from the root node to leaf node (result label) base on the input features

To make our decision tree reusable, I will use OOP for implementing the tree. Comments will be included in the code to explain integral part of the algorithm

# Node class

In [8]:
class Node:
    def __init__(
        self,
        feature=None,
        threshold=None,
        left=None,
        right=None,
        value=None,
    ):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        return self.value is not None # node contains the prediction (or in this case, the most common label)

# Decision Tree class

In [9]:
from collections import Counter


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=10, criterion='information_gain'):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.criterion = criterion
        self.root = None

    # splitting criteria scoring functions
    @staticmethod
    def _shannon_entropy(y):
        num_entries = len(y)
        label_counts = Counter(y)
        shannon_ent = 0.0

        for label in label_counts:
            prob = float(label_counts[label]) / num_entries
            shannon_ent -= prob * np.log2(prob)
        return shannon_ent

    def _information_gain(self, y, left_sub_tree, right_sub_tree):
        base_entropy = self._shannon_entropy(y)

        num_entries = len(y)
        num_left_entries = len(left_sub_tree)
        num_right_entries = len(right_sub_tree)

        left_entropy = self._shannon_entropy(y.iloc[left_sub_tree])
        right_entropy = self._shannon_entropy(y.iloc[right_sub_tree])

        child_entropy = (num_left_entries/num_entries) * left_entropy + \
            (num_right_entries/num_entries) * right_entropy

        return base_entropy - child_entropy

    @staticmethod
    def _gini(y):
        gini_index = 1
        label_occurences = Counter(y)
        total = len(y)

        for label in label_occurences:
            p = label_occurences[label]/total
            gini_index -= np.square(p)

        return gini_index

    def _gini_index(self, y, left_sub_tree, right_sub_tree):
        num_entries = len(y)
        num_left_entries = len(left_sub_tree)
        num_right_entries = len(right_sub_tree)

        left_gini = self._gini(y.iloc[left_sub_tree])
        right_gini = self._gini(y.iloc[right_sub_tree])
        return (num_left_entries/num_entries) * left_gini + \
            (num_right_entries/num_entries) * right_gini

    def _gain_ratio(self, y, left_sub_tree, right_sub_tree):
        intrinsic_value = 0

        num_entries = len(y)
        num_left_entries = len(left_sub_tree)
        num_right_entries = len(right_sub_tree)

        for n in [num_left_entries, num_right_entries]:
            if n > 0:
                proportion = n/num_entries
                intrinsic_value -= proportion * np.log2(proportion)

        if intrinsic_value == 0:
            return 0
        return self._information_gain(y, left_sub_tree, right_sub_tree) / intrinsic_value

    def _calculate_criteria(self, X, y, threshold):
        # Split the tree first base on the current threshold
        left_sub_tree, right_sub_tree = self._split(X, threshold)

        match self.criterion:
            case 'gini':
                return self._gini_index(y, left_sub_tree, right_sub_tree)
            case 'gain_ratio':
                return self._gain_ratio(y, left_sub_tree, right_sub_tree)
            case 'information_gain':
                return self._information_gain(y, left_sub_tree, right_sub_tree)

    # Function to build the tree recursively
    def _generate_tree(self, X, y, cur_depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
 
        # edge case, where there's no label in y
        if(len(y) == 0):
            return Node(value=None)
            
        # Early stopping 
        if (
            cur_depth >= self.max_depth or 
            n_labels == 1 or
            n_samples < self.min_samples_split
        ):
            most_common_label = Counter(y).most_common(1)[0][0]
            return Node(value=most_common_label)

        # Creating a random group for each recursive tree splitting
        random_feature = np.random.choice(
            n_features, n_features, replace=False)

        # Find best split from parent
        best_split_idx, best_split_threshold = self._get_best_split(
            X, y, random_feature)

        # Recursively build the left subtree and right subtree
        left_sub_tree, right_sub_tree = self._split(
            X.iloc[:, best_split_idx], best_split_threshold)

        left = self._generate_tree(
            X.iloc[left_sub_tree, :], y.iloc[left_sub_tree], cur_depth + 1)

        right = self._generate_tree(
            X.iloc[right_sub_tree, :], y.iloc[right_sub_tree], cur_depth + 1)

        return Node(best_split_idx, best_split_threshold, left, right)

    def _get_best_split(self, X, y, random_features):
        max_criteria = float('-inf')
        min_criteria = float('+inf')

        best_split_idx = None
        best_split_threshold = None
        for feature_idx in random_features:
            X_feature = X.iloc[:, feature_idx]
            thresholds = np.unique(X_feature)

            for threshold in thresholds:
                # Calculate the criteria for splitting
                criteria = self._calculate_criteria(X_feature, y, threshold)

                # Updating the best split
                # If gini index, the smaller the better
                if (self.criterion == 'gini' and criteria < min_criteria):
                    min_criteria = criteria
                    best_split_idx = feature_idx
                    best_split_threshold = threshold
                    continue

                # else if model used information gain and gain ratio criteria, the higher the better
                if (criteria > max_criteria):
                    max_criteria = criteria
                    best_split_idx = feature_idx
                    best_split_threshold = threshold

        return best_split_idx, best_split_threshold

    # Perform binary split the tree base on the given threshold ((A < v) or (A ≥ v))
    def _split(self, X, threshold):
        # group indices that are smaller than the split threshold
        left_sub_tree = np.argwhere(X <= threshold).flatten()

        # group indices that are larger than the threshold
        right_sub_tree = np.argwhere(X > threshold).flatten()
        return left_sub_tree, right_sub_tree

    # Tree traversal function
    def _dfs(self, x, node: Node):
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._dfs(x, node.left)
        return self._dfs(x, node.right)

    def fit(self, X, y):
        self.root = self._generate_tree(X, y)

    def predict(self, X):
        predictions = []
        for _, x in X.iterrows():
            predictions.append(self._dfs(x, self.root))

        return np.array(predictions)


## Testing the decision tree models

As a result, I implemented a reusuable decision tree that can be train with 3 different split criterias

### Decision tree using information gain

In [10]:
clf = DecisionTree(criterion='information_gain')
clf.fit(X_train, y_train)

In [11]:
y_pred = clf.predict(X_test)

In [12]:
accuracy = np.sum(y_test == y_pred) / len(y_test)
print(accuracy)

0.9672067901234568


### Decision tree using gini index

In [13]:
clf = DecisionTree(criterion='gini')
clf.fit(X_train, y_train)

In [14]:
y_pred = clf.predict(X_test)

In [15]:
accuracy = np.sum(y_test == y_pred) / len(y_test)
print(accuracy)

0.9488811728395061


### Decision tree using gain ratio

In [16]:
clf = DecisionTree(criterion='gain_ratio')
clf.fit(X_train, y_train)

In [17]:
y_pred = clf.predict(X_test)

In [18]:
accuracy = np.sum(y_test == y_pred) / len(y_test)
print(accuracy)

0.9660493827160493


## Ensemble of the three to create random forrest

An ensemble method for decision trees also known as random forrest. Random forrest will utilise multiple decision tree (which making it a 'forrest'), and training each decision tree with random subset of data (hence 'random')

For this Random Forrest model, I will compile 3 decision trees with different criterion. During prediction, all 3 decision tree will predict the input data. For each predicted value from each row, the most common prediction will be chose as the final prediction for that row

In [19]:
class RandomForrest:
    def __init__(self, min_samples_split=2, max_depth=10):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        criterions = ['gain_ratio', 'information_gain', 'gini']
        for criteria in criterions:
            tree = DecisionTree(
                criterion=criteria,
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth
            )
            X_sample, y_sample = self._sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    @staticmethod
    def _sample(X, y):
        n_rows, _ = X.shape
        samples = np.random.choice(n_rows, n_rows, replace=True)
        return X.iloc[samples], y.iloc[samples]

    def predict(self, X):
        preds = np.array([tree.predict(X) for tree in self.trees]) # get the predictions from all 3 trees
        predictions = []
        for row_num in range(X.shape[0]):
            row_pred = [pred[row_num] for pred in preds]
            predictions.append(Counter(row_pred).most_common(1)[0][0])
        return np.array(predictions)


In [20]:
random_forrest = RandomForrest()
random_forrest.fit(X_train, y_train)

In [21]:
y_pred = random_forrest.predict(X_test)

In [22]:
accuracy = np.sum(y_test == y_pred) / len(y_test)
print(accuracy)

0.964891975308642


Overall, the random forrest model achieve a very good accuracy of ~96%. This could potentially improve by doing the following:
- Hyperparameter tuning the classifier, where we can experiment with different variables until the desired accuracy is reached
- Tree pruning, other than limiting the minimum sample size for each split, or limiting the tree height we can stop splitting current node if the entropy does not improve by some pre-defined preset (impurity threshold).
    - For the implementation of this. I figure we can return the best criteria score in the `_get_best_split()` method in the decision tree. After that `_generate_tree()` will make an early stopping if the criteria score is below a certain threshold
